# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True) # Rename the column
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "", regex=True) # Get rid of square brackets in team names
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "", regex=True)
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "", regex=True)
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "", regex=True)

league = 'NHL'

"""
Prepares a dataframe with the name of the teams for each metropolitan area
"""
def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', league]]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    # set index to be Metropolitan Area
    teams.set_index('Metropolitan area', inplace=True)
    # save original indecies
    or_indecies = teams.index
    # rename areas to match team names
    teams = teams.rename(index={"New York City": "New York", 
                                "Washington, D.C.": "Washington", 
                                "San Francisco Bay Area": "San Jose",
                                "Tampa Bay Area": "Tampa Bay",
                                "Denver": "Colorado",
                                "Minneapolis–Saint Paul": "Minnesota",
                                "Dallas–Fort Worth": "Dallas",
                                "Miami–Fort Lauderdale": "Florida",
                                "Phoenix": "Arizona",
                                "Las Vegas": "Vegas",
                                "Raleigh": "Carolina"})

    teams = teams.reset_index()
    # add area names to the team names
    teams['team'] = teams['Metropolitan area'] + " " +  teams['team']

    # rename particular instances (Why are Devils say New Jersey when they are in New York???)
    teams.iloc[29, 1] = 'New Jersey Devils'
    teams.iloc[30, 1] = 'Anaheim Ducks'
    teams.iloc[31, 1] = 'New York Islanders'
    
    return teams, or_indecies

"""
Calculates the correlation by merging the teams dataframe with wins/losses
"""
def nhl_correlation():
    nhl_df = pd.read_csv("assets/nhl.csv")
    
    # take only year 2018
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    # get rid of * at the end of team names
    nhl_df['team'] = nhl_df['team'].str.replace(r"\*", "", regex=True)
    # get only team name, number of wins, number of losses
    nhl_df = nhl_df[['team', 'W', 'L']]

    # keep only actual teams, drop region names (Central Division, etc.)
    nhl_df = nhl_df[nhl_df['team'] != nhl_df['W']]

    # convert column dtypes for futher processing
    nhl_df = nhl_df.astype({'team': str, 'W': int, 'L': int})
    # calculate win-loss ratio
    nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    
    # get the teams df and the original indecies
    teams, original_indecies = prepare_teams_df()
    
    # merge teams with their wins and losses
    merged_df = pd.merge(teams, nhl_df, how='outer', on='team')
    merged_df['Metropolitan area'] = original_indecies
    
    # get rid of those that have no w/l available
    merged_df = merged_df.dropna()

    # calculate
    merged_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209881

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [6]:
league = 'NBA'

"""
Prepares a dataframe with the name of the teams for each metropolitan area
"""
def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})
    
    teams['team'] = teams['team'].str.strip()

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', league]]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    # set index to be Metropolitan Area
    teams.set_index('Metropolitan area', inplace=True)
    # save original indecies
    or_indecies = teams.index
    # rename areas to match team names
    teams = teams.rename(index={"New York City": "New York", 
                                "Washington, D.C.": "Washington", 
                                "San Francisco Bay Area": "Golden State",
                                "Tampa Bay Area": "Tampa Bay",
                                "Indianapolis": "Indiana",
                                "Salt Lake City": "Utah",
                                "Minneapolis–Saint Paul": "Minnesota",
                                "Dallas–Fort Worth": "Dallas",
                                "Miami–Fort Lauderdale": "Miami",
                                })

    teams = teams.reset_index()
    # add area names to the team names
    teams['team'] = teams['Metropolitan area'] + " " +  teams['team']

    teams.iloc[28, 1] = 'Brooklyn Nets'
        
    return teams, or_indecies

"""
Calculates the correlation by merging the teams dataframe with wins/losses
"""
def nba_correlation():    
    nba_df = pd.read_csv("assets/nba.csv")
    
    # take only year 2018
    nba_df = nba_df[nba_df['year'] == 2018]
    
    # get rid of * and () in team names
    nba_df['team'] = nba_df['team'].str.replace(r"\*", "", regex=True)
    nba_df['team'] = nba_df['team'].str.replace(r"\(.*\)", "", regex=True)
    nba_df['team'] = nba_df['team'].str.strip()

    # get only team name, number of wins, number of losses
    nba_df = nba_df[['team', 'W', 'L']]
    
    # keep only actual teams, drop region names (Central Division, etc.)
    nba_df = nba_df[nba_df['team'] != nba_df['W']]
    
    # convert column dtypes for further processing
    nba_df = nba_df.astype({'team': str, 'W': int, 'L': int})
    # calculate win-loss ratio
    nba_df['W/L%'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    
    teams, original_indecies = prepare_teams_df()
    
    # merge teams with their w/l%
    merged_df = pd.merge(teams, nba_df, how='outer', on='team')
    merged_df['Metropolitan area'] = original_indecies
    
    # get rid of those that have no w/l available
    merged_df = merged_df.dropna()
    
    # calculate
    merged_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844611

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [22]:
league = 'MLB'

"""
Prepares a dataframe with the name of the teams for each metropolitan area
"""
def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})
    
    teams['team'] = teams['team'].str.strip()

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', league]]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    # set index to be Metropolitan Area
    teams.set_index('Metropolitan area', inplace=True)
    # save original indecies
    or_indecies = teams.index
    # rename areas to match team names
    teams = teams.rename(index={"New York City": "New York", 
                                "Washington, D.C.": "Washington", 
                                "San Francisco Bay Area": "San Francisco",
                                "Tampa Bay Area": "Tampa Bay",
                                "Denver": "Colorado",
                                "Phoenix": "Arizona",
                                "Indianapolis": "Indiana",
                                "Salt Lake City": "Utah",
                                "Minneapolis–Saint Paul": "Minnesota",
                                "Dallas–Fort Worth": "Texas",
                                "Miami–Fort Lauderdale": "Miami",
                                })

    teams = teams.reset_index()
    # add area names to the team names
    teams['team'] = teams['Metropolitan area'] + " " +  teams['team']
    
    teams.iloc[29, 1] = "Oakland Athletics"
        
    return teams, or_indecies

def mlb_correlation():     
    mlb_df=pd.read_csv("assets/mlb.csv")

    # take only year 2018
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    # strip white spaces
    mlb_df['team'] = mlb_df['team'].str.strip()

    # get only team name, number of wins, number of losses
    mlb_df = mlb_df[['team', 'W', 'L']]
    
    # keep only actual teams, drop region names (Central Division, etc.)
    mlb_df = mlb_df[mlb_df['team'] != mlb_df['W']]
    
    # convert column dtypes for further processing
    mlb_df = mlb_df.astype({'team': str, 'W': int, 'L': int})
    # calculate win-loss ratio
    mlb_df['W/L%'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    
    teams, original_indecies = prepare_teams_df()
    
    # merge teams with their w/l%
    merged_df = pd.merge(teams, mlb_df, how='outer', on='team')
    merged_df['Metropolitan area'] = original_indecies
    
    # get rid of those that have no w/l available
    merged_df = merged_df.dropna()
    
    # calculate
    merged_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [70]:
league = 'NFL'

"""
Prepares a dataframe with the name of the teams for each metropolitan area
"""
def prepare_teams_df():
    # divide team names into separate teams (because they were given as one string)
    teams = cities[league].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    # add the name of the area column
    teams['Metropolitan area'] = cities['Metropolitan area']
    # change df from wide to long
    teams = pd.melt(teams, id_vars=['Metropolitan area'])
    # drop a useless column
    teams = teams.drop(columns=['variable'])
    # change empty cells into np.nan
    teams = teams.replace("", np.nan).replace("—", np.nan).dropna()
    # reset index to default (because before pd.melt it was a different dataframe)
    teams = teams.reset_index()
    # drop the index column from resetting the index
    teams = teams.drop(columns=['index'])
    # rename the last column
    teams = teams.rename(columns={"value": "team"})
    
    teams['team'] = teams['team'].str.strip()

    # merge two dataframes to get the population column
    teams = pd.merge(teams, cities, how="left", on="Metropolitan area")
    # get only necessary columns (team, Population, NFL)
    teams = teams[['Metropolitan area', 'team', 'Population', league]]
    teams = teams.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    
    # set index to be Metropolitan Area
    teams.set_index('Metropolitan area', inplace=True)
    # rename areas to match team names
    teams = teams.rename(index={"New York City": "New York", 
                                "Washington, D.C.": "Washington", 
                                "San Francisco Bay Area": "San Francisco",
                                "Las Vegas": "San Francisco",
                                "Boston": "New England",
                                "Minneapolis–Saint Paul": "Minnesota",
                                "Phoenix": "Arizona",
                                "Dallas–Fort Worth": "Dallas",
                                "Miami–Fort Lauderdale": "Miami",
                                "Tampa Bay Area": "Tampa Bay",
                                "Nashville": "Tennessee",
                                "Charlotte": "Carolina"
                                })
    
    # save original indecies
    or_indecies = teams.index

    teams = teams.reset_index()
    # add area names to the team names
    teams['team'] = teams['Metropolitan area'] + " " +  teams['team']
            
    teams.iloc[5, 1] = 'Washington Redskins'
    teams.iloc[24, 1] = 'Oakland Raiders'
    teams.drop([32], inplace=True)
    
    return teams, or_indecies[:32]

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    
    # take only year 2018
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    # get rid of * and () in team names
    nfl_df['team'] = nfl_df['team'].str.replace(r"\*", "", regex=True)
    nfl_df['team'] = nfl_df['team'].str.replace(r"\+", "", regex=True)
    nfl_df['team'] = nfl_df['team'].str.replace(r"\(.*\)", "", regex=True)
    nfl_df['team'] = nfl_df['team'].str.strip()

    # get only team name, number of wins, number of losses
    nfl_df = nfl_df[['team', 'W', 'L']]
    
    # keep only actual teams, drop region names (Central Division, etc.)
    nfl_df = nfl_df[nfl_df['team'] != nfl_df['W']]
    
    # convert column dtypes for further processing
    nfl_df = nfl_df.astype({'team': str, 'W': int, 'L': int})
    # calculate win-loss ratio
    nfl_df['W/L%'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    
    teams, original_indecies = prepare_teams_df()
    
    # merge teams with their w/l%
    merged_df = pd.merge(teams, nfl_df, how='outer', on='team')
    merged_df['Metropolitan area'] = original_indecies
    
    # get rid of those that have no w/l available
    merged_df = merged_df.dropna()
    
    # calculate
    merged_df = merged_df.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    
    population_by_region = merged_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return merged_df
    #return stats.pearsonr(population_by_region, win_loss_by_region)

nfl_correlation()

,W/L%,Population
Metropolitan area,,
Arizona,0.187500,4661537
Atlanta,0.437500,5789700
Baltimore,0.625000,2798886
Buffalo,0.375000,1132804
Carolina,0.437500,2474314
Chicago,0.750000,9512999
Cincinnati,0.375000,2165139
Cleveland,0.466667,2055612
Dallas,0.625000,7233323


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values